In [19]:
import importlib

from motor.motor_asyncio import AsyncIOMotorClient
from pymongo.server_api import ServerApi
import bokeh.models
import bokeh.plotting
import bokeh.io


import sp_project.modelling.modelling
import sp_project.db_access.db_entsoe as db_a


bokeh.io.output_notebook()

ConfigurationError: The resolution lifetime expired after 21.608 seconds: Server 192.168.1.1 UDP port 53 answered The DNS operation timed out.; Server fdaa:bbcc:ddee:0:3a35:fbff:fe15:633f UDP port 53 answered The DNS operation timed out.; Server 192.168.1.1 UDP port 53 answered The DNS operation timed out.; Server fdaa:bbcc:ddee:0:3a35:fbff:fe15:633f UDP port 53 answered The DNS operation timed out.; Server 192.168.1.1 UDP port 53 answered The DNS operation timed out.; Server fdaa:bbcc:ddee:0:3a35:fbff:fe15:633f UDP port 53 answered The DNS operation timed out.; Server 192.168.1.1 UDP port 53 answered The DNS operation timed out.; Server fdaa:bbcc:ddee:0:3a35:fbff:fe15:633f UDP port 53 answered The DNS operation timed out.; Server 192.168.1.1 UDP port 53 answered The DNS operation timed out.; Server fdaa:bbcc:ddee:0:3a35:fbff:fe15:633f UDP port 53 answered The DNS operation timed out.

In [10]:
importlib.reload(db_a)

<module 'sp_project.db_access.db_access_entsoe' from '/Users/missd/Library/Mobile Documents/com~apple~CloudDocs/ZHAW/4. Semester/4. Scientific Programming/Project/src/sp_project/db_access/db_access_entsoe.py'>

In [11]:
df_raw = await db_a.extract_energy_data_raw()
df_daily = await db_a.extract_energy_data_daily()

In [18]:
import bokeh.layouts # import column
import bokeh.models # import ColumnDataSource, RangeTool
import bokeh.plotting # import figure, show
import bokeh.palettes

raw_source = bokeh.models.ColumnDataSource(data=df_raw)
day_source = bokeh.models.ColumnDataSource(data=df_daily)


p = bokeh.plotting.figure(
    height=300, 
    width=800, 
    x_axis_type="datetime", 
    x_axis_location="above", 
    # tools="xpan", 
    # toolbar_location=None,
    x_range=(df_raw.index[0], df_raw.index[-1]),
)



p.yaxis.axis_label = 'Energy Production [MW]'

select = bokeh.plotting.figure(
    title="Drag the middle and edges of the selection box to change the range above",
    height=130, 
    width=800, 
    y_range=p.y_range,
    x_axis_type="datetime",
    y_axis_type=None,
    tools="", 
    toolbar_location=None, 
)


range_tool = bokeh.models.RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2


for col_name, color in zip(df_raw.columns, bokeh.palettes.Colorblind[df_raw.shape[1]]):
    p.line('datetime', col_name, source=raw_source, line_color=color)
    select.line('date', col_name, source=day_source, line_color=color)

select.ygrid.grid_line_color = None
select.add_tools(range_tool)

fig = bokeh.layouts.column(p, select)

bokeh.plotting.show(fig)

In [16]:
import bokeh.embed
import json

with open("test.json", "wt", encoding="UTF-8") as fh:
    json.dump(bokeh.embed.json_item(fig), fh)

In [10]:
p.xaxis.formatter.days

'%m/%d'

In [1]:
if False:
    import matplotlib.pyplot as plt

    fig,axes = plt.subplots(4,1,figsize=(16,16),sharex=True)
    for ax,(k,v) in zip(axes,all_data.items()):
        v.plot(ax=ax)
        ax.set_ylabel(k)
    all_data.temp_C.resample("D").mean().plot(ax=axes[0])
    all_data.temp_C.resample("D").min().plot(ax=axes[0])
    all_data.temp_C.resample("D").max().plot(ax=axes[0])